## CLASE 1 ML for finance
---

Medidas de performance:
1. MSE
2. L1-Loss
3. Error rate

Separamos al mundo del ML en:
1. **Supervised learning**: Tenemos una guía y el *training set* se compone de pares (*features* y *labels*)
2. **Unsupervised learning**: No existe guía y el *training set* no posee *labels*. De alguna forma, el algoritmo debe aprender un proceso subyacente en los datos que incluso nosotros no sabemos cuál es. 
3. **Reinforcement learning**: Existe guía y el *training set* no posee *labels*, pero existe un mecanismo de feedback parcial o *reward*. 

![image.png](attachment:image.png)

Ahora, el campo de las finanzas tiene algunos problemas que en general no se encuentra en otros: 1) los datos no son iid ni estacionarios, 2) el ratio señal ruido es bajísimo, 3) el campo de accionar suele ser muy amplio y hay alta incertidumbre y 4) la interpretabilidad de los resultados es importante. 


## CLASE 1 - Workflow modelo con scikit-learn

***
### __0. Inicialización__

### <p align="center">Imports</p>

In [ ]:
import numpy as np
import sklearn.model_selection as sk_ms
import sklearn.linear_model as sk_lm
import sklearn.svm as sk_sv

### <p align="center">Carga de datos</p>

En este caso crearemos en forma manual datos sintéticos (veremos luego que sklearn tiene helpers para esto). __Típicamente__ aquí se importan de alguna fuente externa.

In [ ]:
rng = np.random.default_rng(1234)  # Seed number 1234
X = rng.uniform(size=(1000, 5))
y = 2. + X @ np.array([1., 3., -2., 7., 5.]) + np.random.normal(size=1000)

***
### __1. Creación de Training y Testing Sets__

Se fija el `random_state` en `1234` para obtener siempre el mismo train-test set. Sacar para su uso real.

In [ ]:
X_tr, X_te, y_tr, y_te = sk_ms.train_test_split(X, y, test_size=.1,
                                                random_state=1234)

***
### __2. Definición de los modelos__

In [ ]:
lr = sk_lm.LinearRegression()  # Modelo Lineal
svr = sk_sv.SVR(gamma='scale')  # Support Vector Regression

***
### __3. Comparación de performance utilizando técnicas de muestreo (ej. Cross-Validation)__

Uso para esto el training set.

In [ ]:
print('Regresión Lineal')
lr_cv = sk_ms.cross_val_score(lr, X_tr, y_tr, cv=5)
print('CV-scores:', lr_cv)
print(f'CV-scores Mean = {lr_cv.mean():.4f}')

print('\nSupport Vector Regression')
svr_cv = sk_ms.cross_val_score(svr, X_tr, y_tr, cv=5)
print('CV-scores:',svr_cv)
print(f'CV-scores Mean = {svr_cv.mean():.4f}')

***
### __4. Evaluación FINAL del modelo ganador sobre el Testing Set__

Este paso __NO__ implica volver a estimar todo si el poder predictivo es malo, sino simplemente entender cómo se comporta nuestro mejor modelo out-of-sample. Si la performance no es la deseada lamentablemente no debería ser aplicado. Habría que seguir investigando y __ESPERAR__ la existencia de nuevos datos para volver a aplicar el workflow, de lo contrario estaríamos haciendo overfitting.

In [ ]:
lr.fit(X_tr, y_tr)
print(f'Testing Set Score : {lr.score(X_te, y_te):.2%}')

***
### __5. Entrenamiento final sobre Dataset completo para puesta en producción__

__Sólo__ se realiza si la performance de 4. cumple con el umbral deseado.

In [ ]:
lr.fit(X, y)

print(f'Coeficientes = {lr.coef_}')
print(f'Intercepto = {lr.intercept_}')

***
### __6. Uso en producción__

In [ ]:
print('Predicción para [1., 4., 6., 0., -2.] = ', end='')
print(lr.predict([[1., 4., 6., 0., -2.]]))